In [1]:
from __future__ import print_function
%matplotlib inline
import mdtraj as md
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import pandas as pd
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')
import MDAnalysis as mda
from statistics import mode
import ast
import msmexplorer as msme
from msmbuilder.utils import load,dump
import itertools
from msmbuilder.featurizer import ContactFeaturizer
from msmbuilder.featurizer import DihedralFeaturizer
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
#XRD Ensemble
#28 4NPQ
#18 4HFI
import seaborn as sns
import ptitprince as pt
sns.set(style="whitegrid",font_scale=1.4)
import matplotlib.collections as clt

In [2]:
import json

In [3]:
traj_notes =  [
              '4HFI_pH46_md4','4HFI_pH46_md5','4HFI_pH46_md6',
        '4HFI_pH46_md7','4HFI_pH46_md8','4HFI_pH46_md9',
                '4HFI_F238L_pH46_md7','4HFI_F238L_pH46_md8','4HFI_F238L_pH46_md9',
        '4HFI_F238L_pH46_md4','4HFI_F238L_pH46_md5','4HFI_F238L_pH46_md6',
              '4HFI_I233T_pH46_md4','4HFI_I233T_pH46_md5','4HFI_I233T_pH46_md6',
        '4HFI_I233T_pH46_md7','4HFI_I233T_pH46_md8','4HFI_I233T_pH46_md9',
                '4HFI_F238LI233T_pH46_md4','4HFI_F238LI233T_pH46_md5','4HFI_F238LI233T_pH46_md6',
            '4HFI_F238LI233T_pH46_md7','4HFI_F238LI233T_pH46_md8','4HFI_F238LI233T_pH46_md9',
            '5NJY_pH70_md1','5NJY_pH70_md2','5NJY_pH70_md3',
              '5NJY_F238L_pH70_md1','5NJY_F238L_pH70_md2','5NJY_F238L_pH70_md3',
              '5NJY_I233T_pH70_md4','5NJY_I233T_pH70_md2','5NJY_I233T_pH70_md3',
              '5NJY_F238LI233T_pH70_md1','5NJY_F238LI233T_pH70_md2','5NJY_F238LI233T_pH70_md3',
              '5NJY_pH46_md1','5NJY_pH46_md2','5NJY_pH46_md3',
              '5NJY_F238L_pH46_md1','5NJY_F238L_pH46_md2','5NJY_F238L_pH46_md3',
              '5NJY_I233T_pH46_md1','5NJY_I233T_pH46_md2','5NJY_I233T_pH46_md3',
              '5NJY_F238LI233T_pH46_md1','5NJY_F238LI233T_pH46_md2','5NJY_F238LI233T_pH46_md3',
                '4HFI_F238A_pH46_md1','4HFI_F238A_pH46_md2','4HFI_F238A_pH46_md3'
                ]


In [4]:
def make_intra_inter_flare(traj_note):
    for interaction in ['hb','sb','pc','ps','ts','hp']:
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.tmd.json', 'r') as f:
            flare = json.load(f)
            intra_flare={'edges': []}
            inter_flare={'edges': []}
            intra_flare_dataframe = pd.DataFrame(columns=['frames', 'name1','name2'])
            inter_flare_dataframe = pd.DataFrame(columns=['frames', 'name1','name2'])

            for i in range (0,len(flare['edges'])):
                if (flare['edges'][i]['name1'][0] == flare['edges'][i]['name2'][0]):
               # & (flare['edges'][i]['name1'][-3] != ':') & (flare['edges'][i]['name1'][-3] != '3') & (flare['edges'][i]['name2'][-3] != ':') & (flare['edges'][i]['name2'][-3] != '3') :
                    flare['edges'][i]['name1'] = flare['edges'][i]['name1'][6:]
                    flare['edges'][i]['name2'] = flare['edges'][i]['name2'][6:]
#                    if int(flare['edges'][i]['name1']) <= 198:
#                        flare['edges'][i]['color'] = 'black'
#                    elif int(flare['edges'][i]['name1']) <= 220:
#                        flare['edges'][i]['color'] = 'red'
#                    elif int(flare['edges'][i]['name1']) <= 245:
#                        flare['edges'][i]['color'] = 'blue'
#                    elif int(flare['edges'][i]['name1']) <= 265:
#                        flare['edges'][i]['color'] = 'green'
#                    else:
#                        flare['edges'][i]['color'] = 'purple'

                    if int(flare['edges'][i]['name1']) == 263:
                        flare['edges'][i]['color'] = 'blue'
                    if int(flare['edges'][i]['name1']) == 200:
                        flare['edges'][i]['color'] = 'red'
                    if int(flare['edges'][i]['name1']) == 239:
                        flare['edges'][i]['color'] = 'green'
                    if int(flare['edges'][i]['name1']) == 243:
                        flare['edges'][i]['color'] = 'purple'
                    if int(flare['edges'][i]['name2']) == 263:
                        flare['edges'][i]['color'] = 'blue'
                    if int(flare['edges'][i]['name2']) == 200:
                        flare['edges'][i]['color'] = 'red'
                    if int(flare['edges'][i]['name2']) == 239:
                        flare['edges'][i]['color'] = 'green'
                    if int(flare['edges'][i]['name2']) == 243:
                        flare['edges'][i]['color'] = 'purple'
                    if intra_flare_dataframe[(intra_flare_dataframe['name1'] == flare['edges'][i]['name1']) & (intra_flare_dataframe['name2'] == flare['edges'][i]['name2'])].shape[0] == 0:
                        intra_flare['edges'].append(flare['edges'][i])
                        intra_flare_dataframe = intra_flare_dataframe.append(flare['edges'][i],ignore_index=True)
                    else:
                        intra_flare['edges'][intra_flare_dataframe[(intra_flare_dataframe['name1'] == flare['edges'][i]['name1']) & (intra_flare_dataframe['name2'] == flare['edges'][i]['name2'])].index[0]]['frames'].extend(flare['edges'][i]['frames'])

                elif (flare['edges'][i]['name1'][0] != flare['edges'][i]['name2'][0]) :
                    #& (flare['edges'][i]['name1'][-3] != ':') & (flare['edges'][i]['name1'][-3] != '3') & (flare['edges'][i]['name2'][-3] != ':') & (flare['edges'][i]['name2'][-3] != '3') :
                    flare['edges'][i]['name1'] = flare['edges'][i]['name1'][6:]
                    flare['edges'][i]['name2'] = flare['edges'][i]['name2'][6:]
#                    if int(flare['edges'][i]['name1']) <= 198:
#                        flare['edges'][i]['color'] = 'black'
#                    elif int(flare['edges'][i]['name1']) <= 220:
#                        flare['edges'][i]['color'] = 'red'
#                    elif int(flare['edges'][i]['name1']) <= 245:
#                        flare['edges'][i]['color'] = 'blue'
#                    elif int(flare['edges'][i]['name1']) <= 265:
#                        flare['edges'][i]['color'] = 'green'
#                    else:
#                        flare['edges'][i]['color'] = 'purple'
                    if int(flare['edges'][i]['name1']) == 263:
                        flare['edges'][i]['color'] = 'blue'
                    if int(flare['edges'][i]['name1']) == 200:
                        flare['edges'][i]['color'] = 'red'
                    if int(flare['edges'][i]['name1']) == 239:
                        flare['edges'][i]['color'] = 'green'
                    if int(flare['edges'][i]['name1']) == 243:
                        flare['edges'][i]['color'] = 'purple'
                    if int(flare['edges'][i]['name2']) == 263:
                        flare['edges'][i]['color'] = 'blue'
                    if int(flare['edges'][i]['name2']) == 200:
                        flare['edges'][i]['color'] = 'red'
                    if int(flare['edges'][i]['name2']) == 239:
                        flare['edges'][i]['color'] = 'green'
                    if int(flare['edges'][i]['name2']) == 243:
                        flare['edges'][i]['color'] = 'purple'
            
                    if inter_flare_dataframe[(inter_flare_dataframe['name1'] == flare['edges'][i]['name1']) & (inter_flare_dataframe['name2'] == flare['edges'][i]['name2'])].shape[0] == 0:
                        inter_flare['edges'].append(flare['edges'][i])
                        inter_flare_dataframe = inter_flare_dataframe.append(flare['edges'][i],ignore_index=True)
                    else:
                        inter_flare['edges'][inter_flare_dataframe[(inter_flare_dataframe['name1'] == flare['edges'][i]['name1']) & (inter_flare_dataframe['name2'] == flare['edges'][i]['name2'])].index[0]]['frames'].extend(flare['edges'][i]['frames'])
        for i in range (0,len(intra_flare['edges'])):
            intra_flare['edges'][i]['frames'] = list(set(intra_flare['edges'][i]['frames']))
        for i in range (0,len(inter_flare['edges'])):
            inter_flare['edges'][i]['frames'] = list(set(inter_flare['edges'][i]['frames']))

        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.inter.tmd.json', 'w') as f:
            json.dump(inter_flare, f)
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.intra.tmd.json', 'w') as f:
            json.dump(intra_flare, f)
        




In [8]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(make_intra_inter_flare)(traj_note) for traj_note in traj_notes)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
ls

In [162]:
def recover_freq_chain(traj_note):
    for interaction in ['hb','sb','pc','ps','ts','hp']:
        freq_data = pd.read_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.tsv',sep='\s+',header=None,skiprows=2,names=['residue1','residue2','contact_freq'])
        freq_data['chain1'] = freq_data['residue1'].apply(lambda x: x[x.find(':')-1])
        freq_data['chain2'] = freq_data['residue2'].apply(lambda x: x[x.find(':')-1])
        freq_data['residue1'] = freq_data['residue1'].apply(lambda x: x[x.find(':')+5:])
        freq_data['residue2'] = freq_data['residue2'].apply(lambda x: x[x.find(':')+5:])
        freq_data.to_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.chain.tsv')

In [186]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(recover_freq_chain)(traj_note) for traj_note in traj_notes)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [233]:
def make_intra_inter_freq(traj_note):
    for interaction in ['hb','sb','pc','ps','ts','hp']:
        freq_data = pd.read_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.tsv',sep='\s+',header=None,skiprows=2,names=['residue1','residue2','contact_freq'])
#        freq_data[freq_data.residue1.apply(lambda x: x[x.find(':')-1]) == freq_data.residue2.apply(lambda x: x[x.find(':')-1])].to_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.intra.tsv')
#        freq_data[freq_data.residue1.apply(lambda x: x[x.find(':')-1]) != freq_data.residue2.apply(lambda x: x[x.find(':')-1])].to_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.inter.tsv')
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.intra.tsv', 'w') as output_file:
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.tsv').readlines()[0])
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.tsv').readlines()[1])

            for index,(res1, res2, frequency) in freq_data[freq_data.residue1.apply(lambda x: x[x.find(':')-1]) == freq_data.residue2.apply(lambda x: x[x.find(':')-1])][['residue1','residue2','contact_freq']].iterrows():
                output_file.write('\t'.join([res1, res2, "%.3f" % frequency]) + "\n")
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.inter.tsv', 'w') as output_file:
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.tsv').readlines()[0])
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.tsv').readlines()[1])
            for index,(res1, res2, frequency) in freq_data[freq_data.residue1.apply(lambda x: x[x.find(':')-1]) != freq_data.residue2.apply(lambda x: x[x.find(':')-1])][['residue1','residue2','contact_freq']].iterrows():
                output_file.write('\t'.join([res1, res2, "%.3f" % frequency]) + "\n")
        
        

In [234]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(make_intra_inter_freq)(traj_note) for traj_note in traj_notes)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [359]:
traj_notes = ['4HFI_pH46_ethanol_md1','4HFI_pH46_ethanol_md2','4HFI_pH46_ethanol_md3',
                '4HFI_F238L_pH46_ethanol_md1','4HFI_F238L_pH46_ethanol_md4','4HFI_F238L_pH46_ethanol_md5','4HFI_F238L_pH46_ethanol_md6',
                '4HFI_F238LI233T_pH46_ethanol_md1','4HFI_F238LI233T_pH46_ethanol_md2','4HFI_F238LI233T_pH46_ethanol_md3',
                '4HFI_F238LI233T_pH46_ethanol_md4','4HFI_F238LI233T_pH46_ethanol_md5','4HFI_F238LI233T_pH46_ethanol_md6',
                '4HFI_I233T_pH46_ethanol_md1','4HFI_I233T_pH46_ethanol_md2','4HFI_I233T_pH46_ethanol_md3']

In [333]:
def make_ethanol_flare(traj_note):
    for interaction in ['hp','hb']:
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.lig.json', 'r') as f:
            flare_frames = pd.DataFrame(columns=['resid','frames'])

            flare = json.load(f)
            for i in range (0,len(flare['edges'])):
                flare['edges'][i]['name1'] = 'X' + flare['edges'][i]['name1'][1:]
                flare['edges'][i]['name2'] = 'X:LIG:0'
                if flare['edges'][i]['name1'] in list(flare_frames['resid']):

        #            flare_frames.loc[flare_frames.resid == flare['edges'][i]['name1'],'frames'].values[0].extend([0])
                    frames_list = flare_frames.loc[flare_frames.resid == flare['edges'][i]['name1'],'frames'].values[0]
                    frames_list.extend(flare['edges'][i]['frames'])
                    flare_frames.loc[flare_frames.resid == flare['edges'][i]['name1'],'frames'].values[0] = frames_list
        #            flare_frames.loc[flare_frames.resid == flare['edges'][i]['name1'],'frames'] = flare_frames.loc[flare_frames.resid == flare['edges'][i]['name1'],'frames'].values.extend(flare['edges'][i]['frames'])
                else:
                    flare_frames = flare_frames.append({'resid':flare['edges'][i]['name1'], 'frames':flare['edges'][i]['frames']}, ignore_index=True)
            for i in range (0,len(flare['edges'])):
                flare['edges'][i]['frames'] = flare_frames[flare_frames.resid == flare['edges'][i]['name1']]['frames'].values[0]        
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.lig.ethanol.json', 'w') as f:
            json.dump(flare, f)

In [338]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(make_ethanol_flare_chain)(traj_note) for traj_note in traj_notes)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [336]:
def make_ethanol_flare_chain(traj_note):
    for interaction in ['hp','hb']:
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.lig.json', 'r') as f:
            flare_frames = pd.DataFrame(columns=['resid','frames'])

            flare = json.load(f)
            for i in range (0,len(flare['edges'])):
                flare['edges'][i]['name2'] = 'X:LIG:0'
                if flare['edges'][i]['name1'] in list(flare_frames['resid']):

        #            flare_frames.loc[flare_frames.resid == flare['edges'][i]['name1'],'frames'].values[0].extend([0])
                    frames_list = flare_frames.loc[flare_frames.resid == flare['edges'][i]['name1'],'frames'].values[0]
                    frames_list.extend(flare['edges'][i]['frames'])
                    flare_frames.loc[flare_frames.resid == flare['edges'][i]['name1'],'frames'].values[0] = frames_list
        #            flare_frames.loc[flare_frames.resid == flare['edges'][i]['name1'],'frames'] = flare_frames.loc[flare_frames.resid == flare['edges'][i]['name1'],'frames'].values.extend(flare['edges'][i]['frames'])
                else:
                    flare_frames = flare_frames.append({'resid':flare['edges'][i]['name1'], 'frames':flare['edges'][i]['frames']}, ignore_index=True)
            for i in range (0,len(flare['edges'])):
                flare['edges'][i]['frames'] = flare_frames[flare_frames.resid == flare['edges'][i]['name1']]['frames'].values[0]        
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.lig.ethanol.chain.json', 'w') as f:
            json.dump(flare, f)

In [357]:
def make_ethanol_freq(traj_note):
    for interaction in ['hb', 'hp']:
        freq_data = pd.read_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.lig.tsv',sep='\s+',header=None,skiprows=2,names=['residue1','residue2','contact_freq'])
        freq_data.residue2 = 'X:LIG:0'
        freq_data.residue1 = 'X' + freq_data['residue1'].apply(lambda x: x[x.find(':'):])
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.lig.new.tsv', 'w') as output_file:
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.tsv').readlines()[0])
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.tsv').readlines()[1])

            for index,(res1, res2, frequency) in freq_data[['residue1','residue2','contact_freq']].iterrows():
                output_file.write('\t'.join([res1, res2, "%.3f" % frequency]) + "\n")
       
        
        

In [360]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(make_ethanol_freq)(traj_note) for traj_note in traj_notes)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [375]:
def make_ethanol_freq_resid_stack(traj_note):
    for interaction in ['hb', 'hp']:
        new_freq_data = pd.DataFrame(columns=['residue1','residue2','contact_freq'])
        freq_data = pd.read_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.lig.new.tsv',sep='\s+',header=None,skiprows=2,names=['residue1','residue2','contact_freq'])
        residues = set(freq_data.residue1)
        for residue in residues:
            new_freq_data = new_freq_data.append({'residue1':residue,'residue2':'X:LIG:0','contact_freq':sum(freq_data[freq_data.residue1==residue]['contact_freq'])},ignore_index=True)
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.lig.new.resid.tsv', 'w') as output_file:
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.tsv').readlines()[0])
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.tsv').readlines()[1])

            for index,(res1, res2, frequency) in new_freq_data[['residue1','residue2','contact_freq']].iterrows():
                output_file.write('\t'.join([res1, res2, "%.3f" % frequency]) + "\n")
       
        
        

In [376]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(make_ethanol_freq_resid_stack)(traj_note) for traj_note in traj_notes)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [378]:
def make_ethanol_freq_resid_stack_40(traj_note):
    for interaction in ['hb', 'hp']:
        new_freq_data = pd.DataFrame(columns=['residue1','residue2','contact_freq'])
        freq_data = pd.read_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.lig.40.tsv',sep='\s+',header=None,skiprows=2,names=['residue1','residue2','contact_freq'])
        residues = set(freq_data.residue1)
        for residue in residues:
            new_freq_data = new_freq_data.append({'residue1':residue,'residue2':'X:LIG:0','contact_freq':sum(freq_data[freq_data.residue1==residue]['contact_freq'])},ignore_index=True)
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.lig.resid.40.tsv', 'w') as output_file:
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.lig.40.tsv').readlines()[0])
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.lig.40.tsv').readlines()[1])

            for index,(res1, res2, frequency) in new_freq_data[['residue1','residue2','contact_freq']].iterrows():
                output_file.write('\t'.join([res1, res2, "%.3f" % frequency]) + "\n")
       
        
        

In [379]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(make_ethanol_freq_resid_stack_40)(traj_note) for traj_note in traj_notes)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [457]:
def make_freq_resid_stack(traj_note):
    for interaction in ['hb','sb','pc','ps','ts','hp']:
        freq_data = pd.read_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.inter.tsv',sep='\s+',header=None,skiprows=2,names=['residue1','residue2','contact_freq'])
#        freq_data[freq_data.residue1.apply(lambda x: x[x.find(':')-1]) == freq_data.residue2.apply(lambda x: x[x.find(':')-1])].to_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.intra.tsv')
#        freq_data[freq_data.residue1.apply(lambda x: x[x.find(':')-1]) != freq_data.residue2.apply(lambda x: x[x.find(':')-1])].to_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.inter.tsv')
        freq_data.residue1 = 'X' + freq_data['residue1'].apply(lambda x: x[x.find(':'):])
        freq_data.residue2 = 'X' + freq_data['residue2'].apply(lambda x: x[x.find(':'):])

        new_freq_data = pd.DataFrame(columns=['residue1','residue2','contact_freq'])

        residues_pairs = set((list[0] + '_' + list[1]) for list in np.asarray(freq_data))
        for residue_pair in residues_pairs:
            new_freq_data = new_freq_data.append({'residue1':residue_pair[:residue_pair.find('_')],'residue2':residue_pair[residue_pair.find('_')+1:],'contact_freq':sum(freq_data[(freq_data.residue1 == residue_pair[:residue_pair.find('_')]) & (freq_data.residue2 == residue_pair[residue_pair.find('_')+1:]) ]['contact_freq'])/5},ignore_index=True)    
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.inter.resid.tsv', 'w') as output_file:
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.inter.tsv').readlines()[0])
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.inter.tsv').readlines()[1])
            for index,(res1, res2, frequency) in new_freq_data[['residue1','residue2','contact_freq']].iterrows():
                output_file.write('\t'.join([res1, res2, "%.3f" % frequency]) + "\n")

        freq_data = pd.read_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.intra.tsv',sep='\s+',header=None,skiprows=2,names=['residue1','residue2','contact_freq'])
#        freq_data[freq_data.residue1.apply(lambda x: x[x.find(':')-1]) == freq_data.residue2.apply(lambda x: x[x.find(':')-1])].to_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.intra.tsv')
#        freq_data[freq_data.residue1.apply(lambda x: x[x.find(':')-1]) != freq_data.residue2.apply(lambda x: x[x.find(':')-1])].to_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.inter.tsv')
        freq_data.residue1 = 'X' + freq_data['residue1'].apply(lambda x: x[x.find(':'):])
        freq_data.residue2 = 'X' + freq_data['residue2'].apply(lambda x: x[x.find(':'):])
        new_freq_data = pd.DataFrame(columns=['residue1','residue2','contact_freq'])

    
        residues_pairs = set((list[0] + '_' + list[1]) for list in np.asarray(freq_data))
        for residue_pair in residues_pairs:
            new_freq_data = new_freq_data.append({'residue1':residue_pair[:residue_pair.find('_')],'residue2':residue_pair[residue_pair.find('_')+1:],'contact_freq':sum(freq_data[(freq_data.residue1 == residue_pair[:residue_pair.find('_')]) & (freq_data.residue2 == residue_pair[residue_pair.find('_')+1:]) ]['contact_freq'])/5},ignore_index=True)    
        with open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.intra.resid.tsv', 'w') as output_file:
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.intra.tsv').readlines()[0])
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.intra.tsv').readlines()[1])
            for index,(res1, res2, frequency) in new_freq_data[['residue1','residue2','contact_freq']].iterrows():
                output_file.write('\t'.join([res1, res2, "%.3f" % frequency]) + "\n")
        
        

In [458]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(make_freq_resid_stack)(traj_note) for traj_note in traj_notes)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
make_freq_resid_rep_avg

In [7]:
traj_notess =  [
              ['4HFI_pH46_md4','4HFI_pH46_md5','4HFI_pH46_md6'],
        ['4HFI_F238L_pH46_md4','4HFI_F238L_pH46_md5','4HFI_F238L_pH46_md6'],
              ['4HFI_I233T_pH46_md4','4HFI_I233T_pH46_md5','4HFI_I233T_pH46_md6'],
            ['4HFI_F238LI233T_pH46_md7','4HFI_F238LI233T_pH46_md8','4HFI_F238LI233T_pH46_md9']]


In [17]:
traj_notess =  [
              ['4HFI_pH46_md4','4HFI_pH46_md5','4HFI_pH46_md6'],
        ['4HFI_pH46_md7','4HFI_pH46_md8','4HFI_pH46_md9'],
                ['4HFI_F238L_pH46_md7','4HFI_F238L_pH46_md8','4HFI_F238L_pH46_md9'],
        ['4HFI_F238L_pH46_md4','4HFI_F238L_pH46_md5','4HFI_F238L_pH46_md6'],
              ['4HFI_I233T_pH46_md4','4HFI_I233T_pH46_md5','4HFI_I233T_pH46_md6'],
        ['4HFI_I233T_pH46_md7','4HFI_I233T_pH46_md8','4HFI_I233T_pH46_md9'],
                ['4HFI_F238LI233T_pH46_md4','4HFI_F238LI233T_pH46_md5','4HFI_F238LI233T_pH46_md6'],
            ['4HFI_F238LI233T_pH46_md7','4HFI_F238LI233T_pH46_md8','4HFI_F238LI233T_pH46_md9'],
            ['5NJY_pH70_md1','5NJY_pH70_md2','5NJY_pH70_md3'],
              ['5NJY_F238L_pH70_md1','5NJY_F238L_pH70_md2','5NJY_F238L_pH70_md3'],
              ['5NJY_I233T_pH70_md4','5NJY_I233T_pH70_md2','5NJY_I233T_pH70_md3'],
              ['5NJY_F238LI233T_pH70_md1','5NJY_F238LI233T_pH70_md2','5NJY_F238LI233T_pH70_md3'],
              ['5NJY_pH46_md1','5NJY_pH46_md2','5NJY_pH46_md3'],
              ['5NJY_F238L_pH46_md1','5NJY_F238L_pH46_md2','5NJY_F238L_pH46_md3'],
              ['5NJY_I233T_pH46_md1','5NJY_I233T_pH46_md2','5NJY_I233T_pH46_md3'],
              ['5NJY_F238LI233T_pH46_md1','5NJY_F238LI233T_pH46_md2','5NJY_F238LI233T_pH46_md3'],
                ['4HFI_F238A_pH46_md1','4HFI_F238A_pH46_md2','4HFI_F238A_pH46_md3']
                ]


In [15]:
def make_freq_resid_rep_avg(traj_notes):
    for interaction in ['hb','sb','pc','ps','ts','hp']:
        new_inter_freq_data = pd.DataFrame(columns=['residue1','residue2','contact_freq'])

        for traj_note in traj_notes:
            freq_data = pd.read_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.inter.resid.tsv',sep='\s+',header=None,skiprows=2,names=['residue1','residue2','contact_freq'])
#                residues_pairs = set((list[0] + '_' + list[1]) for list in np.asarray(freq_data))
            for index, row in freq_data.iterrows():
#            for residue_pair in residues_pairs:
                if new_inter_freq_data[(new_inter_freq_data.residue1 == row['residue1']) & (new_inter_freq_data.residue2 == row['residue2'])].shape[0] == 0:
                    new_inter_freq_data = new_inter_freq_data.append({'residue1':row['residue1'],'residue2':row['residue2'],'contact_freq':row['contact_freq']/len(traj_notes)},ignore_index=True)
                else:
                    new_inter_freq_data.loc[(new_inter_freq_data.residue1 == row['residue1']) & (new_inter_freq_data.residue2 == row['residue2']),'contact_freq'] += row['contact_freq']/len(traj_notes)
        with open('/home/scottzhuang/pdc/'+ traj_notes[0] +'/getcontact/'+ traj_notes[0] +'.'+ interaction +'.freq.inter.resid.rep_avg.tsv', 'w') as output_file:
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_notes[0] +'/getcontact/'+ traj_notes[0] +'.'+ interaction +'.freq.inter.tsv').readlines()[0])
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_notes[0] +'/getcontact/'+ traj_notes[0] +'.'+ interaction +'.freq.inter.tsv').readlines()[1])
            for index,(res1, res2, frequency) in new_inter_freq_data[['residue1','residue2','contact_freq']].iterrows():
                output_file.write('\t'.join([res1, res2, "%.3f" % frequency]) + "\n")

        new_intra_freq_data = pd.DataFrame(columns=['residue1','residue2','contact_freq'])

        for traj_note in traj_notes:
            freq_data = pd.read_csv('/home/scottzhuang/pdc/'+ traj_note +'/getcontact/'+ traj_note +'.'+ interaction +'.freq.intra.resid.tsv',sep='\s+',header=None,skiprows=2,names=['residue1','residue2','contact_freq'])
#                residues_pairs = set((list[0] + '_' + list[1]) for list in np.asarray(freq_data))
            for index, row in freq_data.iterrows():
#            for residue_pair in residues_pairs:
                if new_intra_freq_data[(new_intra_freq_data.residue1 == row['residue1']) & (new_intra_freq_data.residue2 == row['residue2'])].shape[0] == 0:
                    new_intra_freq_data = new_intra_freq_data.append({'residue1':row['residue1'],'residue2':row['residue2'],'contact_freq':row['contact_freq']/len(traj_notes)},ignore_index=True)
                else:
                    new_intra_freq_data.loc[(new_intra_freq_data.residue1 == row['residue1']) & (new_intra_freq_data.residue2 == row['residue2']),'contact_freq'] += row['contact_freq']/len(traj_notes)
        with open('/home/scottzhuang/pdc/'+ traj_notes[0] +'/getcontact/'+ traj_notes[0] +'.'+ interaction +'.freq.intra.resid.rep_avg.tsv', 'w') as output_file:
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_notes[0] +'/getcontact/'+ traj_notes[0] +'.'+ interaction +'.freq.intra.tsv').readlines()[0])
            output_file.write(open('/home/scottzhuang/pdc/'+ traj_notes[0] +'/getcontact/'+ traj_notes[0] +'.'+ interaction +'.freq.intra.tsv').readlines()[1])
            for index,(res1, res2, frequency) in new_intra_freq_data[['residue1','residue2','contact_freq']].iterrows():
                output_file.write('\t'.join([res1, res2, "%.3f" % frequency]) + "\n")
        
        

In [18]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(make_freq_resid_rep_avg)(traj_notes) for traj_notes in traj_notess)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]